In [1]:
!pip install constants

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of constants to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
  Created wheel for constants: filename=constants-0.6.0-py3-none-any.whl size=5458 sha256=d1e9fb4c52c7d66cb21dc2e246b67e4459513237d495500feac3b662e69a089d
  Stored in directory: /Users/mansisingh/Library/Caches/pip/wheels/45/16/b2/58f34beef901270143f1d215138cc4e3cd086bcc24a9839173
Successfully built constants


In [13]:
!pip install PyPDF2

In [3]:
!pip install transformers

In [4]:
import pickle
import nltk
import re
import pandas as pd
import numpy as np
import constants as const
import PyPDF2
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer
import torch
nltk.download(['punkt', 'stopwords', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mansisingh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mansisingh/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mansisingh/nltk_data...


True

In [5]:
def tfidf_similarity(tokenized_text1, tokenized_text2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([tokenized_text1, tokenized_text2])

    # Similarity Measurement
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    return cosine_sim[0][0]

In [6]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def compute_bert_embeddings(sentence, model, tokenizer):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0)
    with torch.no_grad():
        output = model(input_ids)
    return output.last_hidden_state.mean(dim=1).numpy()

In [8]:
def combine_embeddings(long_text):
  max_segment_length = 512  # Maximum segment length allowed by BERT
  segments = [long_text[i:i + max_segment_length] for i in range(0, len(long_text), max_segment_length)]

  # Initialize an empty tensor for aggregated embeddings
  aggregated_embeddings = torch.zeros((1, 768))  # Ensure the size matches BERT's embedding size (e.g., 768)

  # Process each segment separately and aggregate embeddings
  for segment in segments:
      encoded_input = tokenizer(segment, return_tensors='pt', padding=True, truncation=True, max_length=512)
      with torch.no_grad():
          output = model(**encoded_input)
      segment_embedding = output.last_hidden_state.mean(dim=1)  # Mean pooling over tokens
      aggregated_embeddings += segment_embedding
  return aggregated_embeddings

In [9]:
def bert_similarity(tokenized_text1, tokenized_text2) :
    embeddings1 = combine_embeddings(tokenized_text1)
    embeddings2 = combine_embeddings(tokenized_text2)

    # Calculate cosine similarity
    return cosine_similarity(embeddings1, embeddings2)[0][0]

In [10]:
!pip install fastapi

In [11]:
!pip install pydantic

In [14]:
from fastapi import FastAPI
from pydantic import BaseModel

class Item(BaseModel):
    t1: str
    t2: str

app = FastAPI()

@app.post("/in")
def create(item: Item):
    dt = item.dict()
    text1 = dt['t1']
    text2 = dt['t2']
    b = str(bert_similarity(text1, text2))
    t =  str(tfidf_similarity(text1, text2))
    return {
        "Bert similarity": b, 
        "Tfidf similarity": t
    }

import nest_asyncio
import uvicorn

if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app)

INFO:     Started server process [1526]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:49853 - "POST /in HTTP/1.1" 200 OK
INFO:     127.0.0.1:49988 - "POST /in HTTP/1.1" 200 OK
